#### Importing libraries 

In [1]:
import requests
import datetime
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

#### Web scrapping & Cleaning

In [3]:
lista=[20100101,20110101,20120101,20130101,20140101,20150101,20160101,20170101,20180101,20190101]
lista2=[]

for i in lista:
    url = f'https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/29005?date={i}'
    respuesta = requests.get(url)
    datos = respuesta.json()
    data=pd.DataFrame.from_records(datos)
    
    data['Fecha']=data['Data'].apply(lambda x: [next(iter(d.values())) for d in x])
    data['Año']=data['Data'].apply(lambda x:[d.get('Anyo') for d in x])
    data['Población']=data['Data'].apply(lambda x:[d.get('Valor') for d in x])
    
    data['Fecha']=data['Fecha'].apply(lambda x: re.sub('\]','',str(x)))
    data['Fecha']=data['Fecha'].apply(lambda x: re.sub('\[','',str(x)))
    
    data['Fecha']=data['Fecha'].apply(lambda x: str(x).strip())
    data['Fecha']=pd.to_numeric(data['Fecha'], errors='ignore')
    data['Fecha'].fillna(0, inplace=True)
    data['Fecha'].apply(int)
    
    data['Fecha']=data['Fecha'].apply(lambda x: datetime.date.fromtimestamp(
                      int(x) // 1000))
    
    data['Año']=data['Año'].apply(lambda x: re.sub('\]','',str(x)))
    data['Año']=data['Año'].apply(lambda x: re.sub('\[','',str(x)))
    
    data['Población']=data['Población'].apply(lambda x: re.sub('\]','',str(x)))
    data['Población']=data['Población'].apply(lambda x: re.sub('\[','',str(x)))
        
    data.drop(columns=['Data','FK_Escala','FK_Unidad'])
    
    lista2.append(data)
    
    total=pd.concat(lista2)
    filtering=total['Año']==''
    
    total=total[~filtering]
    total=total.drop(columns=['Data','FK_Escala','FK_Unidad','COD'])

    total['Sexo']= total.Nombre.str.contains(pat = '(Mujeres.)', regex = True) 
    total['Sexo']=total['Sexo'].replace(True,'Mujeres')
    
    total['Sexo']=np.where(total['Nombre'].str.contains('Hombres'), "Hombres", total['Sexo'])
    total['Sexo']=total['Sexo'].replace(False,'Total')
    
    municipios=[]
    for i in total.Nombre.str.split('.'):
        municipios.append(i[0])
    
    total['Nombre']=municipios

/home/carmencuadrado/miniconda3/envs/Ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/home/carmencuadrado/miniconda3/envs/Ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/home/carmencuadrado/miniconda3/envs/Ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/home/carmencuadrado/miniconda3/envs/Ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/home/carmencuadrado/miniconda3/envs/Ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/home/carmencuadrado/miniconda3/envs/Ironhack/lib/pytho

In [6]:
total.shape

(243609, 5)

In [9]:
e=pd.read_excel('/home/carmencuadrado/Ironhack/Final_Project/20codmun.xlsx')

In [10]:
d=pd.read_excel('/home/carmencuadrado/Ironhack/Final_Project/Muniacteco_20-45_ANDALUCIA.xls')

CompDocError: Expected "little-endian" marker, found b'\xff\xff'

In [32]:
d[d['Unnamed: 4'].str.contains('Zuera')]

,Relación de municipios y códigos por comunidades autónomas y provincias a 1 de enero de 2020,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
8129,02,50,298,9,Zuera
